In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
steam = pd.read_csv ('../Datasets/data_Api/function3.csv', usecols=['user_id','item_id','sentiment_analysis','recommend','app_name'])

In [5]:
df = steam
def recomendacion_juego(df, input_item_id, num_recommendations=5):
    # Filtrar el DataFrame para obtener solo las filas relacionadas con el juego de entrada
    input_item = df[df['item_id'] == input_item_id]

    if input_item.empty:
        return "El ID de producto no se encuentra en el conjunto de datos."

    # Extraer el usuario y sus recomendaciones para ese juego
    user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='recommend', fill_value=0)

    # Calcular la similitud coseno entre el juego de entrada y todos los demás juegos
    similarity_scores = cosine_similarity(user_item_matrix.T)

    # Obtener las puntuaciones de similitud para el juego de entrada
    input_item_scores = similarity_scores[input_item_id]

    # Obtener los índices de los juegos más similares (excluyendo el juego de entrada)
    similar_items_indices = input_item_scores.argsort()[:-1][-num_recommendations:]

    # Obtener los IDs y nombres de los juegos más similares
    similar_items = user_item_matrix.columns[similar_items_indices]

    # Obtener nombres de juegos correspondientes a los IDs
    game_names_df = df[df['item_id'].isin(similar_items)][['item_id', 'app_name']]

# Fusionar los DataFrames para obtener nombres correspondientes a los IDs
    merged_df = pd.merge(pd.DataFrame(similar_items, columns=['item_id']), game_names_df, on='item_id')

# Imprimir las recomendaciones de juegos con saltos de línea y números de índice
    print(f"Juegos recomendados similares al juego con ID {input_item_id}:")

    for idx, (item_id, game_name) in enumerate(zip(merged_df['item_id'], merged_df['app_name']), start=1):
        print(f"{idx}. Juego ID: {item_id}, Name: {game_name}")
        

# Ejemplo de uso:
input_item_id = 80
recomendacion_juego(df, input_item_id)

Juegos recomendados similares al juego con ID 80:
1. Juego ID: 40970, Name: Stronghold Crusader HD
2. Juego ID: 40970, Name: Stronghold Crusader HD
3. Juego ID: 40970, Name: Stronghold Crusader HD
4. Juego ID: 40970, Name: Stronghold Crusader HD
5. Juego ID: 40970, Name: Stronghold Crusader HD
6. Juego ID: 232890, Name: Stronghold Crusader 2
7. Juego ID: 232890, Name: Stronghold Crusader 2
8. Juego ID: 232890, Name: Stronghold Crusader 2
9. Juego ID: 232890, Name: Stronghold Crusader 2
10. Juego ID: 209270, Name: Hero Academy
11. Juego ID: 209270, Name: Hero Academy
12. Juego ID: 209270, Name: Hero Academy
13. Juego ID: 330100, Name: BossConstructor
14. Juego ID: 342560, Name: Airships: Conquer the Skies


In [4]:
steam[steam['item_id'] == 214950]

,app_name,user_id,item_id,recommend,sentiment_analysis
27777,Total War™: ROME II - Emperor Edition,76561198057086148,214950,1,1
27778,Total War™: ROME II - Emperor Edition,76561198026040814,214950,1,0
27779,Total War™: ROME II - Emperor Edition,76561198088408984,214950,1,0
27780,Total War™: ROME II - Emperor Edition,76561198009498722,214950,0,1
27781,Total War™: ROME II - Emperor Edition,76561198066084140,214950,1,2
...,...,...,...,...,...
27859,Total War™: ROME II - Emperor Edition,76561198047679159,214950,1,2
27860,Total War™: ROME II - Emperor Edition,76561198059323447,214950,0,2
27861,Total War™: ROME II - Emperor Edition,jordywt,214950,1,1
27862,Total War™: ROME II - Emperor Edition,76561198073784330,214950,1,2


Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [8]:
steam

,app_name,user_id,item_id,recommend,sentiment_analysis
0,Half-Life,EizanAratoFujimaki,70,1,1
1,Half-Life,GamerFag,70,1,1
2,Half-Life,76561198020928326,70,1,1
3,Half-Life,Bluegills,70,1,2
4,Half-Life,76561198071955492,70,1,1
...,...,...,...,...,...
43422,Counter-Strike: Condition Zero,76561198023508728,80,0,1
43423,Counter-Strike: Condition Zero,Lone_walker,80,1,2
43424,Counter-Strike: Condition Zero,green290,80,1,2
43425,Counter-Strike: Condition Zero,174gamecuman700kngkakak,80,1,1


In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import pickle
from surprise import accuracy

# Cargar el dataset
steam = steam

# Generar un rating a partir de recommend y sentiment_analysis
label_encoder = LabelEncoder()
steam['user_id_num'] = label_encoder.fit_transform(steam['user_id'])
steam['rating'] = np.where(steam['recommend'] == True,
                                 np.where(steam['sentiment_analysis'] == 2, 5,
                                          np.where(steam['sentiment_analysis'] == 1, 3, 1)),
                                 np.where(steam['sentiment_analysis'] == 2, 4,
                                          np.where(steam['sentiment_analysis'] == 1, 2, 0)))

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 5))

# Cargar el DataFrame en el formato correcto para Surprise
data = Dataset.load_from_df(steam[['user_id_num', 'item_id', 'rating']], reader)

# Separar el dataset en entrenamiento y testeo
train, test = train_test_split(data, test_size=0.25, random_state=42)

# Aplicar GridSearch para encontrar los mejores hiperparámetros
param_grid = {'n_factors': [5, 50, 100], 'n_epochs': [5, 10, 20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
gs.fit(data)
print(f"Mejor RMSE: {gs.best_score['rmse']}")
print(f"Mejores hiperparámetros: {gs.best_params['rmse']}")

# Obtener el modelo con los mejores hiperparámetros
best_model = gs.best_estimator['rmse']

# Definir y entrenar el modelo con los mejores hiperparámetros
model = SVD(n_factors=best_model.n_factors, n_epochs=best_model.n_epochs,
            lr_all=best_model.lr_all, reg_all=best_model.reg_all)
model.fit(train)

# Evaluar el modelo en el conjunto de entrenamiento y testeo
train_predictions = model.test(train.build_testset())
test_predictions = model.test(test)
print(f"RMSE en el conjunto de entrenamiento: {accuracy.rmse(train_predictions)}")
print(f"RMSE en el conjunto de prueba: {accuracy.rmse(test_predictions)}")

# Guardar el modelo en un archivo
with open('./SVD_model.pkl', 'wb') as file:
    pickle.dump(model, file)

    